In [3]:
from pathlib import Path
import duckdb


# ndjson test data from
# https://www.hl7.org/fhir/examples-ndjson.zip
# NOTE: Binary.ndjson and Group.ndjson
# should be renamed (e.g. Binary_, Group_) as these are SQL keywords
EXAMPLES = Path('./input-data/examples')
EXAMPLES_PARQUET = Path("./output/examples-parquet/")
EXAMPLES_DB = Path("./output/examples.duckdb")


In [4]:
# ingest synthea data, storing intermedia parquets as well
with duckdb.connect((EXAMPLES_DB).as_posix()) as conn:
    for file_ in EXAMPLES.glob("*.ndjson"):
        duckdb.read_json(file_.as_posix()).to_parquet((EXAMPLES_PARQUET / f"{file_.stem}.parquet").as_posix())
    for file_ in EXAMPLES_PARQUET.glob("*.parquet"):
        stmt = f"""
            DROP TABLE IF EXISTS {file_.stem};
            CREATE TABLE {file_.stem} AS SELECT * FROM '{file_.as_posix()}'
            """
        conn.execute(stmt)

In [6]:
# read-only
conn = duckdb.connect(database=EXAMPLES_DB.as_posix(), read_only=True)
conn.close()

In [7]:
# same test, with our own data
# TO DO: fails, because schema not inferred properly
for file_ in DATA.glob("intermediate/*.ndjson"):
    duckdb.read_json(file_.as_posix()).to_parquet((DATA / "final" / f"{file_.stem}.parquet").as_posix())

NameError: name 'DATA' is not defined